In [12]:
%matplotlib widget
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import seaborn as sns
import pandas as pd
import itertools
from pathlib import Path
data_dir = Path('data/Torus')

In [2]:
sns.set_style("whitegrid")

In [8]:
from src.modules import Evolve, Torus, Ring, utils, Plotter, metric
from src.modules.data_structure import *

In [9]:
n= 32
m= 32
c_type= data_structure.ConfigType.Torus
size = n*m
p_state= 0.5
seed= 698203
steps=2000
np.random.seed(seed)
pi=np.random.permutation(n)
magnetList = [0.0, 0.2, 0.4, 0.6, 0.8]
rule_set=  lambda x,y: (x+y)%2
pair = lambda k: (k // m, k % m)
ruler = lambda i : Rule.STABLE if i== 1 else Rule.UNSTABLE
rules = [ruler(rule(pair(i)[0], pair(i)[1])) for i in range(size)]
pi=np.random.permutation(n)
metricList= [Metric.SpinGlass, Metric.Magnetization]
samples = 300

agree= {'init_magnet': [], 'length' : [], 'Magnetization_mean':[], 'Magnetization_min':[], 'Magnetization_max':[],
        'Magnetization_std':[], 'SpinGlass':[]}   

conf = utilities.createConfig(c_type, n=n, m=m, rules = rules)

NameError: name 'rule' is not defined

In [52]:
for i, mag in enumerate(magnetList):
    print(i)
    for j in range(samples):
        init_states= [State.ON]*round((size*(0.5 + mag/2))-10**(-9))+ [State.OFF]*round((size*(0.5 -mag/2)+10**(-9)))
        np.random.shuffle(init_states)
        conf.set_states(init_states)
        stables = [0]
        array, out = Evolve.evolve(conf, perm= pi, steps=steps, 
                                   metricList=metricList, cycleBreak=True)
        curr_energy = out[Metric.SpinGlass][-1]
        curr_magnet= out[Metric.Magnetization][-1]
        length, start = utilities.cycle_length(array)


        agree['init_magnet'].append(mag)
        agree['length'].append(length)
        agree['Magnetization_mean'].append(curr_magnet)
        agree['Magnetization_std'].append(np.std(magnetization(array[start: start+length])))
        agree['Magnetization_min'].append(np.min(magnetization(array[start: start+length])))
        agree['Magnetization_max'].append(np.max(magnetization(array[start: start+length])))
        agree['SpinGlass'].append(curr_energy)  


data = pd.DataFrame(agree)

0
1
2
3
4


In [10]:
title = '{2}_{0}_by_{0}_{1}_chess_agreement.csv'.format(32, 3000, 'Torus')
data.to_csv(title, encoding='utf-8', index=False)


NameError: name 'data' is not defined

In [19]:
title = '{2}_{0}_by_{0}_{1}_chess_agreement.csv'.format(32, 2000, 'Torus')
data=pd.read_csv(data_dir/title)

In [24]:
magnet_fix, magnet_ax = plt.subplots()
sns.violinplot(data = data, x= 'init_magnet', y = 'Magnetization_mean', cut=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='init_magnet', ylabel='Magnetization_mean'>

In [21]:
ener_fig, ener_ax = plt.subplots()
sns.violinplot(data = data, x= 'init_magnet', y = 'SpinGlass')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='init_magnet', ylabel='SpinGlass'>

In [23]:
ener_fig, ener_ax = plt.subplots()
sns.violinplot(data = data, x= 'init_magnet', y = 'length', cut=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='init_magnet', ylabel='length'>